In [1]:
import pandas as pd
import os
import time
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from model import LineageMantaObjectOptimize
from utils import create_raw_table, transfrom_column_name_before_create_table, convert_df_to_oracle_format
from config import (NODE_PATH, LINEAGE_SOURCE, LINEAGE_TARGET,
                    EXPORT_MANTA_FILE_PATH, LINEAGE_MANTA_RAW_TEST,
                    LINEAGE_MANTA_RAW_MAIN,LINEAGE_MANTA_FLOW_TEST, 
                    LINEAGE_MANTA_FLOW_MAIN,  TABLE_MANTA_LINEAGE_RAW_COLUMN, 
                    IMPORT_MANTA_FILE_PATH, TABLE_MANTA_LINEAGE_FLOW)
from repo import RepoLineage

os.getcwd(), c:\Users\hungpv6\Documents\GitHub\DG_tool\main


In [2]:
if __name__ == '__main__':
    # start_time = time.time()
    database = 'test'
    enviroment = 'DEV'
    db_connection = RepoLineage()
    db_connect = db_connection.get_connection(db_name='test',environment = 'DEV', user = 'dtc')
    object = LineageMantaObjectOptimize(lineage_name= 'manta')
    df_raw, table_name_list = create_raw_table(object)
    df_graph = None

   

Connection to database 'test' established successfully.
LINEAGE_MANTA_W4
TABLE_NAME =  FM_CLIENT_CONTACT | FILE_NAME=  relations_FM_CLIENT_CONTACT | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_FM_CLIENT_CONTACT.csv
TABLE_NAME =  FM_CLIENT_INDVL | FILE_NAME=  relations_FM_CLIENT_INDVL | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_FM_CLIENT_INDVL.csv
TABLE_NAME =  HDB | FILE_NAME=  relations_HDB | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_HDB.csv
TABLE_NAME =  RB_ACCT | FILE_NAME=  relations_RB_ACCT | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_RB_ACCT.csv
TABLE_NAME =  RB_TRAN_HIST | FILE_NAME=  relations_RB_TRAN_HIST | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_RB_TRAN_HIST.csv


In [3]:
for table_name in table_name_list:
    # time.sleep(5)
    df = df_raw.query(f"Table_name == '{table_name}'")
    if len(df) >= 1:
        df_describe_op = object.describe_table_optimized(
            df_path=df,
            source_col=LINEAGE_SOURCE,
            target_col=LINEAGE_TARGET,
        )
        all_values = list(df_describe_op.get("all_values"))
        redact_node_df = object.redact_node_with_int(total_nodes_df=df.copy())
        combo_list = redact_node_df[NODE_PATH].tolist()
        # combo_list = [1,2,1868,72,824,928,40,121,366,1691,35,548,663
        # ,1944,509,638,1800,620,750,54,1509,2166,1531,1173,1013,74,345,245
        # ,762,2165,1439,1525,75,631,840,1734,1738,2001,728,1728,1987,1341,145,1724]
        dictionary_op = object.mapping_dict_nodepath_optimize(df =redact_node_df)
        adjacency_list = object.create_adjacency_list_optimized(dictionary = dictionary_op)
        graph = object.deploy_lineage_manta_optimized(df_raw= df.copy(),
                                    character_split = ' ==> ',
                                    combo_list_copy=combo_list, 
                                    dictionary=dictionary_op,
                                    adjacency_list = adjacency_list, 
                                    source_col=LINEAGE_SOURCE, 
                                    target_col=LINEAGE_TARGET)
        object.add_name_of_table_column(df = graph, table_name = table_name)
        if df_graph is None:
            df_graph = graph
        else:
            df_graph = pd.concat([df_graph, graph])
    else:
        print(f"Table {table_name} is empty")


Table HDB is empty


In [4]:
object.export_to_csv(df=df_graph, file_path=f'{EXPORT_MANTA_FILE_PATH}/df_graph.csv', index = False)
# print(df_graph)

In [5]:
# df_graph_unpivoted = object.unpivoted_column(df=df_graph, column_unpivoted='Flow_implement_raw', unpivoted_column_name='unpivoted_column',character_split= ' ==> ')
df_graph_extract_node = object.extract_components_optimized(table = df_graph, column_extract='Raw_Node', column_name_extracted_list = [
            {'table_name_extract':-2},
            {'column_name': -1},
            {'database_type': 0},
            {'database_name': 1},
            {'schema': 2}
        ])
# df_graph_extract_node_relations = object.extract_components_optimized(table = df_graph_extract_node, column_extract='unpivoted_column',  column_name_extracted_list = [
#             {'table_name_relations':-2},
#             {'column_name_relations': -1},
#             {'database_type_relations': 0},
#             {'database_name_relations': 1},
#             {'schema_relations': 2}
#         ])



In [6]:
df_graph_final = transfrom_column_name_before_create_table(df=df_graph_extract_node)   
pd.set_option('display.max_rows', None)

In [7]:
df_graph_final.columns

Index(['FLOW_RAW', 'NODE_OBJECT', 'VALUE_OBJECT_TYPE', 'TABLE_NAME_CSV',
       'TABLE_NAME_NODE_OBJECT', 'COLUMN_NAME_OBJECT', 'SOURCE_DATABASE_TYPE',
       'SOURCE_DATABASE_NAME', 'SCHEMA_OBJECT', 'UPDATE_TIME'],
      dtype='object')

In [8]:
# import pandas as pd
# import numpy as np

# def extract_components_optimized(table, column_extract, character_split='.', column_name_extracted_list=None):
#     """
#     Extracts specific components from a column in a DataFrame using vectorized operations.

#     Parameters:
#         table (pd.DataFrame): The input DataFrame.
#         column_extract (str): The column from which components will be extracted.
#         character_split (str, optional): The delimiter for splitting. Defaults to '.'.
#         column_name_extracted_list (list, optional): List of dicts mapping column names to index positions.

#     Returns:
#         pd.DataFrame: Updated DataFrame with extracted components.
#     """
#     if column_name_extracted_list is None:
#         column_name_extracted_list = [
#             {'table_name':-2},
#             {'column_name': -1},
#             {'database_type': 0},
#             {'database_name': 1},
#             {'schema': 2}
#         ]

#     # Tách dữ liệu thành list
#     split_series = table[column_extract].astype(str).str.split(character_split)

#     # Tạo các cột mới với giá trị mặc định là NaN
#     for item in column_name_extracted_list:
#         column_name, index_position = next(iter(item.items()))
        
#         # Gán giá trị dựa trên điều kiện: Nếu dòng có đủ phần tử thì lấy giá trị, nếu không thì NaN
#         table[column_name] = split_series.apply(lambda x: x[index_position] if len(x) > abs(index_position) else np.nan)

#     return table
# df = pd.DataFrame({
#     'unpivoted_column': [
#         'Oracle.W4DBN4.HDB.YGACCOUNT.YGACCOUNT_BODY.<6,1>PLSQL_BLOCK.<10,5>SELECT',
#         'Oracle.W4DBN4.HDB.YGACCOUNT.YGACCOUNT_BODY',
#         'Oracle.W4DBN4.HDB'
#     ]
# })

# column_name_extracted_list = [
#     {'table_name_relations': -2},
#     {'column_name_relations': -1},
#     {'database_type_relations': 0},
#     {'database_name_relations': 1},
#     {'schema_relations': 2}
# ]

# df_result = extract_components_optimized(df, 'unpivoted_column', '.', column_name_extracted_list)
# print(df_result)


In [9]:
# print(df_graph.head(100))
# print(len(df_graph))

        

In [10]:
# data = db_connection.create_lineage_manta_table(connection = db_connect,
#                                                 table_name = LINEAGE_MANTA_RAW_MAIN,
#                                                 column_definitions= TABLE_MANTA_LINEAGE_RAW_COLUMN)
# db_connection.insert_data_from_dataframe(connection = db_connect, table_name = LINEAGE_MANTA_RAW_MAIN, dataframe=convert_df_to_oracle_format(transfrom_column_name_before_create_table(df_raw)))

In [11]:
# df_raw.to_csv(f'{EXPORT_MANTA_FILE_PATH}/df_raw.csv', index = False)# 

In [12]:
data = db_connection.create_lineage_manta_table(connection = db_connect,
                                                table_name = LINEAGE_MANTA_FLOW_MAIN,
                                                column_definitions= TABLE_MANTA_LINEAGE_FLOW)
db_connection.insert_data_from_dataframe(connection = db_connect, table_name = LINEAGE_MANTA_FLOW_MAIN, dataframe=convert_df_to_oracle_format((df_graph_final)))

In [13]:
# db_connection.rename_column(connection = db_connect, table_name = LINEAGE_MANTA_FLOW_MAIN, old_column_name = 'SCHEMA', new_column_name = 'SCHEMA_OBJECT')

In [14]:
print(len(df_graph_final))

807996
